In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.io import loadmat
import itertools
from statsmodels.distributions.empirical_distribution import ECDF
import copy
from scipy import stats
import pandas as pd

In [4]:
a = pd.read_csv("files_same.csv")
b = pd.read_csv("files_missing.csv")


In [11]:
a
# len(a.subdirectory.unique())

,subdirectory,filename_a,present_a,filename_b,present_b,present_both
0,cov-sq_EmpiricalCovariance,cov-sq_EmpiricalCovariance~010~000494~networkc...,True,cov-sq_EmpiricalCovariance~010~000494~threshad...,True,True
1,cov-sq_EmpiricalCovariance,cov-sq_EmpiricalCovariance~030~000190~networkc...,True,cov-sq_EmpiricalCovariance~030~000190~threshad...,True,True
2,cov-sq_EmpiricalCovariance,cov-sq_EmpiricalCovariance~036~000312~networkc...,True,cov-sq_EmpiricalCovariance~036~000312~threshad...,True,True
3,cov-sq_EmpiricalCovariance,cov-sq_EmpiricalCovariance~027~000594~networkc...,True,cov-sq_EmpiricalCovariance~027~000594~threshad...,True,True
4,cov-sq_EmpiricalCovariance,cov-sq_EmpiricalCovariance~036~000265~networkc...,True,cov-sq_EmpiricalCovariance~036~000265~threshad...,True,True
...,...,...,...,...,...,...
3419270,pec_orth,pec_orth~100~000577~networkcalcs~lowest~10844.mat,True,pec_orth~100~000577~threshadj~lowest~10844.mat,True,True
3419271,pec_orth,pec_orth~090~000227~networkcalcs~lowest~4368.mat,True,pec_orth~090~000227~threshadj~lowest~4368.mat,True,True
3419272,pec_orth,pec_orth~062~000327~networkcalcs~lowest~5574.mat,True,pec_orth~062~000327~threshadj~lowest~5574.mat,True,True
3419273,pec_orth,pec_orth~075~000236~networkcalcs~lowest~25095.mat,True,pec_orth~075~000236~threshadj~lowest~25095.mat,True,True


In [19]:
b[b['subdirectory'] == "pdist_cosine"]

,subdirectory,filename_a,present_a,filename_b,present_b,present_both
292357,pdist_cosine,NaN,False,pdist_cosine~092~000539~threshadj~highest~4885...,True,False
292358,pdist_cosine,NaN,False,pdist_cosine~100~000579~threshadj~highest~1107...,True,False
292359,pdist_cosine,NaN,False,pdist_cosine~092~000075~threshadj~highest~4714...,True,False
292360,pdist_cosine,NaN,False,pdist_cosine~108~000301~threshadj~highest~5039...,True,False
292361,pdist_cosine,NaN,False,pdist_cosine~081~000337~threshadj~highest~2443...,True,False
...,...,...,...,...,...,...
308383,pdist_cosine,NaN,False,pdist_cosine~090~000209~threshadj~highest~4890...,True,False
308384,pdist_cosine,NaN,False,pdist_cosine~090~000033~threshadj~highest~4884...,True,False
308385,pdist_cosine,NaN,False,pdist_cosine~075~000410~threshadj~highest~2646...,True,False
308386,pdist_cosine,NaN,False,pdist_cosine~108~000453~threshadj~highest~5228...,True,False


In [17]:
b[b['subdirectory'] == "pdist_cosine"].filename_b.values[0]


'pdist_cosine~092~000539~threshadj~highest~4885.mat'

In [2]:
def plot_ecdf(results, m, measure):
    # Create square subplots (adjust figsize if needed)
    fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

    # Plot Binary data and compute KS test
    try:
        data1 = results[m]['BIN_soz_soz']
        data2 = results[m]['BIN_non_non']
        ecdf1 = ECDF(data1)
        ecdf2 = ECDF(data2)
        
        ax[0].plot(ecdf1.x, ecdf1.y, label="soz", color='red')
        ax[0].plot(ecdf2.x, ecdf2.y, label="non", color='blue')
        
        # Two-sample KS test for binary data
        stat, p_val = stats.ks_2samp(data1, data2)
        ax[0].text(0.05, 0.95, f'p = {p_val:.3f}', transform=ax[0].transAxes,
                verticalalignment='top')
    except Exception as e:
        pass

    # Plot Weighted data and compute KS test
    try:
        data1_w = results[m]['WEI_soz_soz']
        data2_w = results[m]['WEI_non_non']
        ecdf1_w = ECDF(data1_w)
        ecdf2_w = ECDF(data2_w)
        
        ax[1].plot(ecdf1_w.x, ecdf1_w.y, label="soz", color='red')
        ax[1].plot(ecdf2_w.x, ecdf2_w.y, label="non", color='blue')
        
        # Two-sample KS test for weighted data
        stat, p_val = stats.ks_2samp(data1_w, data2_w)
        ax[1].text(0.05, 0.95, f'p = {p_val:.3f}', transform=ax[1].transAxes,
                verticalalignment='top')
    except Exception as e:
        pass

    # Set labels and titles for each axis and force square plots
    ax[0].set_xlabel("Binary Value")
    ax[0].set_ylabel("ECDF")
    ax[0].set_title("Binary")
    ax[0].grid(True)
    # ax[0].set_aspect('square', adjustable='box')

    ax[1].set_xlabel("Weighted Value")
    ax[1].set_ylabel("ECDF")
    ax[1].set_title("Weighted")
    ax[1].grid(True)
    # ax[1].set_aspect('square', adjustable='box')

    # Create one common legend at the bottom (single row with 2 columns)
    handles, labels = ax[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, 0.0))

    plt.suptitle(f"{m}")
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])

    output_path = "/home/dan/Downloads/BCT/images"
    plt.savefig(os.path.join(output_path, f"{measure}~{m}.png"))
    plt.close()

In [3]:
# load soz
soz_path = "/media/dan/Data2/calculations/connectivity/additional_calcs/mats/bary-sq_euclidean_max"
files = sorted(os.listdir(soz_path))
soz = {}
for file in tqdm(files):
    if file.endswith(".mat"):
        # file format: base_name + ~{pid}.mat
        # example pdist_euclidean~001.mat
        # extract pid
        base = file.split(".")[0]
        pid = int(base.split("~")[1])
        if pid not in soz:
            soz[pid] = {}
        soz[pid]['soz'] = loadmat(os.path.join(soz_path, file))['soz'][0]



  0%|          | 0/69 [00:00<?, ?it/s]

100%|██████████| 69/69 [00:04<00:00, 14.20it/s]


In [4]:
# break network into soz and non and soz->non/non->soz

for pid in soz:
    s = soz[pid]['soz']
    idxs = np.arange(len(s))
    soz_idx = idxs[s == 1]
    non_idx = idxs[s == 0]

    # all two pair combinations of soz_idx (remove self-edges)
    soz_edges = np.array(list(itertools.combinations(soz_idx, 2)))
    soz_edges = soz_edges[soz_edges[:, 0] != soz_edges[:, 1]]

    # all two pair combinations of non_soz_idx (remove self-edges)
    non_edges = np.array(list(itertools.combinations(non_idx, 2)))
    non_edges = non_edges[non_edges[:, 0] != non_edges[:, 1]]

    # all two pair combinations of soz_idx and non_soz_idx (remove self-edges)
    soz_non_edges = np.array(list(itertools.product(soz_idx, non_idx)))
    soz_non_edges = soz_non_edges[soz_non_edges[:, 0] != soz_non_edges[:, 1]]


    soz[pid]['soz_soz'] = soz_edges
    soz[pid]['non_soz'] = non_edges
    soz[pid]['soz_non'] = soz_non_edges


In [2]:
measures = ['modularity','clustering_coefficient','degrees','strength','betweenness','eigenvector_centrality','participation_coefficient', 'local_efficiency', 'rich_club_coefficient','kcoreness_centrality', 'nodal_eccentricity',]
len(measures)*91

1001

In [103]:
out_dir = "/home/dan/Downloads/BCT/outputs"

directories = list(sorted(os.listdir(out_dir)))
measures = ['modularity','clustering_coefficient','degrees','strength','betweenness','eigenvector_centrality','participation_coefficient', 'local_efficiency', 'rich_club_coefficient','kcoreness_centrality', 'nodal_eccentricity',]
divisions = {'BIN_soz_soz': [],
'BIN_non_non': [],
'WEI_soz_soz': [],
'WEI_non_non': []}

for d in tqdm(directories, desc="Processing directories"):
    if os.path.isdir(os.path.join(out_dir, d)):
        results = {}
        for m in measures:
            results[m] = copy.deepcopy(divisions)
        for f in tqdm(os.listdir(os.path.join(out_dir, d)), desc="Loading results"):
            if f.endswith(".mat"):
                out = loadmat(os.path.join(out_dir, d, f), struct_as_record=False, squeeze_me=True)['out']
                pid = f.split("~")[1]
                pid = int(pid)
                idx = np.arange(len(soz[pid]['soz']))
                soz_idxs = idx[soz[pid]['soz'] == 1]
                non_idxs = idx[soz[pid]['soz'] == 0]

                for m in measures:
                    measure = getattr(out, m)
                    types = [attr for attr in dir(measure) if not attr.startswith('__') and not attr.startswith('_fieldnames')]
                    for t in types:
                        arr = getattr(measure, t)
                        if t == "binary":
                            results[m]['BIN_soz_soz'].extend(arr[soz_idxs])
                            results[m]['BIN_non_non'].extend(arr[non_idxs])
                        elif t == "weighted":
                            results[m]['WEI_soz_soz'].extend(arr[soz_idxs])
                            results[m]['WEI_non_non'].extend(arr[non_idxs])
                        
        for m in tqdm(measures, desc="Plotting ECDFs"):
            plot_ecdf(results, m, d)


Loading results: 100%|██████████| 41549/41549 [00:40<00:00, 1028.46it/s]t]
/home/dan/miniconda3/envs/pyspicalc/lib/python3.12/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/dan/miniconda3/envs/pyspicalc/lib/python3.12/site-packages/matplotlib/cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
Loading results: 100%|██████████| 41546/41546 [00:40<00:00, 1036.20it/s]/it] 
